# 中文姓名與暱稱的性別預測建模
請用機率模型示範中文姓名與暱稱的性別預測建模，並提供資料收集、前處理、特徵工程、模型選擇與訓練、模型評估以及性別預測示範的完整流程。

## 資料收集與載入

取得包含中文姓名/暱稱和對應性別的資料集。


In [15]:
import pandas as pd
import random

data = {'name': ['陳建宏', '林雅婷', '王志豪', '張美珍', '劉俊賢', '黃淑娟', '吳明哲', '徐佩珊', '李宗翰', '蔡雅芳',
                 '楊文傑', '陳淑萍', '許志明', '何美玲', '邱俊豪', '沈雅君', '鄭建國', '曾淑華', '蘇志偉', '羅雅惠',
                 '林俊宏', '王淑芬', '張明德', '李佩玲', '陳志豪', '吳雅雯', '黃俊傑', '蔡淑芳', '劉明憲', '林美君',
                 '楊志強', '陳雅玲', '王俊凱', '張佩珊', '李建華', '黃美華', '吳志明', '徐雅芬', '許俊賢', '何淑君',
                 '鄭明哲', '曾美玲', '蘇建宏', '羅淑萍', '林志偉', '王雅婷', '張俊豪', '李淑華', '陳明德', '吳美珍'],
        'gender': ['男', '女', '男', '女', '男', '女', '男', '女', '男', '女',
                   '男', '女', '男', '女', '男', '女', '男', '女', '男', '女',
                   '男', '女', '男', '女', '男', '女', '男', '女', '男', '女',
                   '男', '女', '男', '女', '男', '女', '男', '女', '男', '女',
                   '男', '女', '男', '女', '男', '女', '男', '女', '男', '女']}


df = pd.DataFrame(data)

# 顯示 DataFrame 的前幾行
print("資料集前幾行:")
display(df.head())

# 顯示資料集的大小
print(f"\n資料集總筆數: {len(df)}")

資料集前幾行:


,name,gender
0,陳建宏,男
1,林雅婷,女
2,王志豪,男
3,張美珍,女
4,劉俊賢,男



資料集總筆數: 50


## 特徵工程

從姓名和暱稱中提取可用於預測性別的特徵。這可能包括姓名長度、特定字元或詞語的頻率等。


In [16]:
# 計算姓名長度
df['name_length'] = df['name'].apply(len)

# 提取名字的部分 (排除姓氏)
# 假設中文姓名通常第一個字是姓氏，這裡提取從第二個字開始的名字部分
df['given_name'] = df['name'].apply(lambda x: x[1:])

# 提取名字部分的第一個字和最後一個字作為特徵
df['given_name_first_char'] = df['given_name'].apply(lambda x: x[0] if len(x) > 0 else '')
df['given_name_last_char'] = df['given_name'].apply(lambda x: x[-1] if len(x) > 0 else '')


# 對名字部分的第一個字和最後一個字進行 One-Hot 編碼
# 注意：這裡使用名字部分進行編碼，不使用姓氏
df_given_name_first_char_one_hot = pd.get_dummies(df['given_name_first_char'], prefix='given_name_first_char')
df_given_name_last_char_one_hot = pd.get_dummies(df['given_name_last_char'], prefix='given_name_last_char')

# 將 One-Hot 編碼後的欄位與原始 DataFrame 合併
# 這裡合併的是基於名字部分的特徵
df = pd.concat([df, df_given_name_first_char_one_hot, df_given_name_last_char_one_hot], axis=1)

# 顯示處理後的 DataFrame 的前幾行和欄位名稱
print("\n處理後的 DataFrame 前幾行:")
display(df.head())
print("\n處理後的 DataFrame 欄位名稱:")
print(df.columns.tolist())


處理後的 DataFrame 前幾行:


,name,gender,name_length,given_name,given_name_first_char,given_name_last_char,given_name_first_char_佩,given_name_first_char_俊,given_name_first_char_宗,given_name_first_char_建,...,given_name_last_char_珊,given_name_last_char_珍,given_name_last_char_翰,given_name_last_char_芬,given_name_last_char_芳,given_name_last_char_華,given_name_last_char_萍,given_name_last_char_豪,given_name_last_char_賢,given_name_last_char_雯
0,陳建宏,男,3,建宏,建,宏,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,林雅婷,女,3,雅婷,雅,婷,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,王志豪,男,3,志豪,志,豪,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,張美珍,女,3,美珍,美,珍,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,劉俊賢,男,3,俊賢,俊,賢,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False



處理後的 DataFrame 欄位名稱:
['name', 'gender', 'name_length', 'given_name', 'given_name_first_char', 'given_name_last_char', 'given_name_first_char_佩', 'given_name_first_char_俊', 'given_name_first_char_宗', 'given_name_first_char_建', 'given_name_first_char_志', 'given_name_first_char_文', 'given_name_first_char_明', 'given_name_first_char_淑', 'given_name_first_char_美', 'given_name_first_char_雅', 'given_name_last_char_偉', 'given_name_last_char_傑', 'given_name_last_char_凱', 'given_name_last_char_君', 'given_name_last_char_哲', 'given_name_last_char_國', 'given_name_last_char_娟', 'given_name_last_char_婷', 'given_name_last_char_宏', 'given_name_last_char_強', 'given_name_last_char_德', 'given_name_last_char_惠', 'given_name_last_char_憲', 'given_name_last_char_明', 'given_name_last_char_玲', 'given_name_last_char_珊', 'given_name_last_char_珍', 'given_name_last_char_翰', 'given_name_last_char_芬', 'given_name_last_char_芳', 'given_name_last_char_華', 'given_name_last_char_萍', 'given_name_last_char_豪', 'given_name_l

## 模型選擇與訓練

選擇一個合適的機率模型 (例如，naivebayes)，並使用處理過的資料集來訓練模型。


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

# 選擇用於模型的特徵 (排除原始姓名、性別、姓氏以及中間處理用的欄位)
# 這裡只使用姓名長度、名字部分第一個字和最後一個字的 One-Hot 編碼特徵
feature_columns = ['name_length'] + [col for col in df.columns if col.startswith('given_name_first_char_') or col.startswith('given_name_last_char_')]
X = df[feature_columns]
y = df['gender']

# 將性別標籤轉換為數值 (例如，'男' -> 0, '女' -> 1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 將資料集分割成訓練集和測試集
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# 選擇一個分類模型 (例如，Logistic Regression 或 Gaussian Naive Bayes)
# model = LogisticRegression(random_state=42)
model = GaussianNB()

# 訓練模型
model.fit(X_train, y_train_encoded)

print("模型訓練完成。")

模型訓練完成。


## 模型評估

使用測試資料來評估模型的效能，例如準確度、精確度、召回率等。


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predict on the test set
y_pred_encoded = model.predict(X_test)

# Calculate evaluation metrics
# 指定 positive class 為 encoding 後的 '女' 的數值標籤 (通常是 1)
# 這裡假設 '女' 被 encoding 為 1，如果不是，需要根據實際 encoding 結果調整
# 可以在訓練前查看 label_encoder.classes_ 和 transform 的結果來確認
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
precision = precision_score(y_test_encoded, y_pred_encoded, pos_label=label_encoder.transform(['女'])[0])
recall = recall_score(y_test_encoded, y_pred_encoded, pos_label=label_encoder.transform(['女'])[0])
f1 = f1_score(y_test_encoded, y_pred_encoded, pos_label=label_encoder.transform(['女'])[0])
conf_matrix = confusion_matrix(y_test_encoded, y_pred_encoded)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9333
Precision: 0.8889
Recall: 1.0000
F1-score: 0.9412
Confusion Matrix:
[[8 0]
 [1 6]]


## 性別預測示範

使用訓練好的模型對新的中文姓名或暱稱進行性別預測示範。


In [25]:
# 1. 準備新的姓名資料
new_names = pd.Series(['趙文哲', '孫雅琳', '周俊明', '吳淑梅', '鄭志強', '王美惠'])

# 2. 應用與訓練資料相同的特徵工程步驟
new_df = pd.DataFrame({'name': new_names})

# 計算姓名長度
new_df['name_length'] = new_df['name'].apply(len)

# 提取名字部分 (排除姓氏)
new_df['given_name'] = new_df['name'].apply(lambda x: x[1:])

# 提取名字部分的第一個字和最後一個字
new_df['given_name_first_char'] = new_df['given_name'].apply(lambda x: x[0] if len(x) > 0 else '')
new_df['given_name_last_char'] = new_df['given_name'].apply(lambda x: x[-1] if len(x) > 0 else '')

# 對新的名字部分的第一個字和最後一個字進行 One-Hot 編碼
new_df_given_name_first_char_one_hot = pd.get_dummies(new_df['given_name_first_char'], prefix='given_name_first_char')
new_df_given_name_last_char_one_hot = pd.get_dummies(new_df['given_name_last_char'], prefix='given_name_last_char')

# 對齊新的 One-Hot 編碼欄位與訓練資料的欄位
# 這確保了即使新資料中出現訓練資料沒有的字，也能正確處理 (欄位值為 0)
# 並且欄位順序與訓練資料一致
all_given_name_first_char_cols = [col for col in X.columns if col.startswith('given_name_first_char_')]
all_given_name_last_char_cols = [col for col in X.columns if col.startswith('given_name_last_char_')]

new_df_given_name_first_char_one_hot = new_df_given_name_first_char_one_hot.reindex(columns=all_given_name_first_char_cols, fill_value=0)
new_df_given_name_last_char_one_hot = new_df_given_name_last_char_one_hot.reindex(columns=all_given_name_last_char_cols, fill_value=0)

# 合併新的特徵
# 這裡只包含姓名長度、名字部分第一個字和最後一個字的 One-Hot 編碼特徵
new_X = pd.concat([new_df[['name_length']], new_df_given_name_first_char_one_hot, new_df_given_name_last_char_one_hot], axis=1)

# 確保 new_X 的欄位順序與訓練資料 X 的欄位順序一致
new_X = new_X[X.columns]


# 3. 使用訓練好的模型進行預測 (預測結果是數值標籤)
predictions_encoded = model.predict(new_X)

# 4. 將數值預測結果轉換回原始性別標籤 ('男', '女')
predicted_genders = label_encoder.inverse_transform(predictions_encoded)

# 5. 顯示預測結果
prediction_results = pd.DataFrame({'姓名': new_names, '預測性別': predicted_genders})
display(prediction_results)

,姓名,預測性別
0,趙文哲,男
1,孫雅琳,女
2,周俊明,男
3,吳淑梅,女
4,鄭志強,男
5,王美惠,女


## 總結與展望

本次中文姓名與暱稱性別預測建模流程如下：

1.  **資料收集與載入**：模擬產生了包含中文姓名和對應性別的資料集，共 50 筆，盡量使用三個字的姓名，並考量了男女生名字中常見的用字。
2.  **資料前處理**：將資料載入 DataFrame，並將性別標籤轉換為數值格式。
3.  **特徵工程**：從姓名中提取姓名長度、名字（排除姓氏）的第一個字和最後一個字作為特徵，並進行 One-Hot 編碼。
4.  **模型選擇與訓練**：選擇了 Gaussian Naive Bayes 模型，並使用處理後的特徵資料訓練了模型。
5.  **模型評估**：使用測試資料評估了模型的效能，計算了準確度、精確度、召回率和 F1-score，並展示了混淆矩陣。
6.  **性別預測示範**：使用訓練好的模型對新的中文姓名進行了性別預測示範。

整個流程展示了如何使用機率模型對中文姓名進行性別預測的建模過程。需要注意的是，這是一個使用小型模擬資料集的範例。
